<a href="https://colab.research.google.com/github/PrajwalGupta3/ML/blob/main/HybridModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update -y
!sudo apt install python3.14
!python3.12 --version

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,351 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,755 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,533 kB]
Hit:13 https://ppa.launc

In [2]:
!pip install torch torchvision scikit-learn numpy --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 931.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 66.5/127.9 MB 124.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.5 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone

In [3]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
from torchvision.models import resnet50
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time

In [4]:
#Set Up Device (Use GPU If Available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet needs 224x224 images
    transforms.ToTensor()           # Convert images to PyTorch tensors
])


In [6]:
# Loading the  CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

100%|██████████| 170M/170M [00:02<00:00, 67.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
#Creating the Data Loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

#Loading the  Pretrained ResNet50 Model
resnet = resnet50(pretrained=True).to(device)
resnet.eval()  # Put model in evaluation mode
print("ResNet50 has been successfully loaded")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 158MB/s]


ResNet50 has been successfully loaded


In [8]:
#This is a function to Extract Features
def extract_features(loader):
    features, labels = [], []
    with torch.no_grad():  #  need not compute gradients
        for images, targets in loader:
            images = images.to(device)  # Move the images to GPU
            feats = resnet(images)  # Extract all features
            features.append(feats.cpu().numpy())  # Move to CPU and  Convert to NumPy(Required by SVM)
            labels.append(targets.numpy())  # Convert labels to NumPy(required by SVM)
    return np.concatenate(features), np.concatenate(labels) #put all the extracted features in a big dataset

In [9]:
#Extracting the features for Training and Testing purpose
print("Extracting the features")
start_time = time.time()

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

end_time = time.time()
print(f"Feature Extraction Completed in {end_time - start_time:.2f} seconds")

Extracting the features
Feature Extraction Completed in 200.66 seconds


In [10]:
#Train the  SVM Classifier on Extracted Features
svm = SVC(kernel='linear')  # Using a linear SVM
svm.fit(train_features, train_labels)
print("SVM Training successfull")



#Test the Model's Performance
preds = svm.predict(test_features)
accuracy = accuracy_score(test_labels, preds)
print(f"Test Accuracy: {accuracy * 100:.2f}% ")

SVM Training successfull
Test Accuracy: 79.72% 


In [14]:
import joblib

#Saving the trained model
joblib.dump(svm, "my_svm_model.pkl")
print("Model is Saved as 'my_svm_model.pkl'")

Model is Saved as 'my_svm_model.pkl'


In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf.fit(train_features, train_labels)

test_acc = rf.score(test_features, test_labels)
print(f"Random Forest Test Accuracy: {test_acc * 100:.2f}%")

Random Forest Test Accuracy: 70.17%


In [13]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=200, use_label_encoder=False, eval_metric="logloss")
xgb.fit(train_features, train_labels)

test_acc = xgb.score(test_features, test_labels)
print(f"XGBoost Test Accuracy: {test_acc * 100:.2f}%")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:04:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Test Accuracy: 77.46%
